In [948]:
%matplotlib inline
import csv
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import datetime as dt
import ast
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler  
from pandas.api.types import is_numeric_dtype
import urllib

In [949]:
%precision 2
# Load data
df = pd.read_csv('params_aarch64_Feb_22_2022.csv')
df1 = pd.read_csv('params_aarch64_Feb_23_2022.csv')
df2 = pd.read_csv('params_aarch64_Feb_25_2022.csv')
df3 = pd.read_csv('params_aarch64_Feb_26_2022.csv')
df4 = pd.read_csv('params_aarch64_Feb_28_2022.csv')
df = pd.concat([df,df1,df2,df3,df4], axis=0)
df.head()

,Time,Voltage,Throttled_Status,Ring_Oscillation,RPI_Model,GPIO_Pin,MAC_Address,Operating_System,Device_Machine,Computer_Name,IP_Address,Uptime,Boottime,CPU_Temp,CPU%,CPU_count,System_Calls,Current_CPU_Freq,Min_CPU_Freq,Max_CPU_Freq,System_Load_Avg_1Min,System_Load_Avg_5Min,System_Load_Avg_15Min,Process_Most_CPU,Physical_Memory,RAM_Usage,Memory%_Availabe,Memory%_Used,Process_Most_Memory,Disk_Usage,Disk_Space,Number_bytes_sent,Number_bytes_received,Number_packets_sent,Number_packets_received
0,2022-02-22 21:04:57.021740,0.926,0x0,2.580,4 Model B 8Gb,11000000000000000000000000,dc:a6:32:d9:d1:56,Linux,aarch64,raspberrypi,192.168.11.12,up 42 minutes,2022-02-22 20:22:14,59.887,26.1,4,0,1800.0,600.0,1800.0,31.25,35.50,29.25,"{'chromium-browse': 38.7, 'python': 32.0, 'tho...",8.192524,545.583104,85.498078,14.5,"{'chromium-browser': 14.66151166325355, 'Xorg'...",9.1,62.528573,3495352,499215165,38724,349508
1,2022-02-22 21:05:32.836466,0.926,0x0,2.580,4 Model B 8Gb,11000000000000000000000000,dc:a6:32:d9:d1:56,Linux,aarch64,raspberrypi,192.168.11.12,up 43 minutes,2022-02-22 20:22:14,61.348,34.5,4,0,1800.0,600.0,1800.0,31.50,35.25,29.50,"{'chromium-browse': 25.700000000000003, 'thonn...",8.192524,545.837056,85.392485,14.6,"{'chromium-browser': 14.533069883527455, 'Xorg...",9.1,62.528573,3496598,499232717,38744,349609
2,2022-02-22 21:05:55.558390,0.926,0x0,2.580,4 Model B 8Gb,11000000000000000000000000,dc:a6:32:d9:d1:56,Linux,aarch64,raspberrypi,192.168.11.12,up 43 minutes,2022-02-22 20:22:14,58.913,1.9,4,0,600.0,600.0,1800.0,24.50,33.25,29.00,"{'chromium-browse': 21.700000000000003, 'thonn...",8.192524,545.898496,85.700415,14.3,"{'chromium-browser': 14.53741960514527, 'Xorg'...",9.1,62.528573,3497154,499244836,38753,349676
3,2022-02-22 21:07:01.069036,0.926,0x0,2.580,4 Model B 8Gb,11000000000000000000000000,dc:a6:32:d9:d1:56,Linux,aarch64,raspberrypi,192.168.11.12,up 44 minutes,2022-02-22 20:22:14,60.861,11.7,4,0,1800.0,600.0,1800.0,18.00,29.75,28.00,"{'pip': 37.9, 'chromium-browse': 19.3, 'thonny...",8.192524,548.749312,84.612385,15.4,"{'chromium-browser': 14.624964002303852, 'Xorg...",9.2,62.528573,3811162,520649503,42376,364168
4,2022-02-22 21:08:01.519968,0.926,0x0,2.579,4 Model B 8Gb,11000000000000000000000000,dc:a6:32:d9:d1:56,Linux,aarch64,raspberrypi,192.168.11.12,up 45 minutes,2022-02-22 20:22:14,63.783,18.6,4,0,1800.0,600.0,1800.0,26.00,29.00,27.75,"{'pip': 34.5, 'chromium-browse': 18.1, 'thonny...",8.192524,582.938624,84.510841,15.5,"{'chromium-browser': 14.625863944707538, 'Xorg...",9.4,62.528573,3907225,523414526,43163,366192


# Data Preprocessing

In [950]:
def clean_column_data_type(df):
    df.columns = [x.lower().strip() for x in df.columns]
    # Convert necessary datatypes to float
    convert_dict_type = {
        'time':'datetime64',
        'voltage':float,
        'ring_oscillation':float,
        'rpi_model':'string',
        'gpio_pin':'string',
        'mac_address':'string',
        'operating_system':'string',
        'device_machine':'string',
        'computer_name':'string',
        'boottime':'datetime64',
        'cpu_temp': float,
        'cpu%': float,
        'cpu_count': int,
        'system_calls': int,
        'current_cpu_freq': int,
        'min_cpu_freq': int,
        'max_cpu_freq': int,
        'system_load_avg_1min': float,
        'system_load_avg_5min': float,
        'system_load_avg_15min': float,
        'physical_memory': float,
        'ram_usage': float,
        'memory%_availabe': float,
        'memory%_used': float,
        'disk_usage': float,
        'disk_space': float,
        'number_bytes_sent': int,
        'number_bytes_received': int,
        'number_packets_sent': int,
        'number_packets_received': int
    }
    df = df.astype(convert_dict_type)

    # round off floats
    for col in df.columns:
        if df[col].dtypes =='float64':
            df[col] = round(df[col],3)
            
    return df

In [951]:
def drop_columns(df, cols):
    df.drop(cols, axis=1, inplace=True)
    return df

In [952]:
def vectorize_cpu_process(df):
    
    # Convert 'process most CPU' to dict
    df['process_most_cpu'] = df['process_most_cpu'].apply(lambda x: ast.literal_eval(x))
    # Vectorize processes' CPU consumption
    vec = DictVectorizer()
    table = vec.fit_transform(df['process_most_cpu']).toarray()
    cpu_process_matrix = pd.DataFrame(data=table,columns = vec.get_feature_names_out())
    """
    Grouping all kworker processes together.
    kworker is a placeholder process for kernel worker threads, which perform most of the actual
    processing for the kernel, especially in cases where there are interrupts, timers, I/O, etc. Handles 
    delayed work from throughout the kernel where the work is not being handled by one of the other queues.
    """
    kworker_cols = [col for col in cpu_process_matrix.columns if 'kworker' in col]
    #print(cpu_process_matrix[kworker_cols].mean(axis=1).mean())
    total_kworker = cpu_process_matrix[kworker_cols].sum(axis=1)
    cpu_process_matrix.drop(kworker_cols,axis=1,inplace=True)
    cpu_process_matrix['kworker'] = total_kworker
    cpu_process_matrix = cpu_process_matrix.add_prefix('cpu_')
    for col in cpu_process_matrix.columns:
        df[col] = cpu_process_matrix[col]
    df.drop(['process_most_cpu'],inplace=True,axis=1)
    
    return df

In [953]:
def vectorize_mem_process(df):
    # Convert 'process most memory' to dict
    df['process_most_memory'] = df['process_most_memory'].apply(lambda x: ast.literal_eval(x))
    # Vectorize processes' Memory consumption
    vec = DictVectorizer()
    table = vec.fit_transform(df['process_most_memory']).toarray()
    memory_process_matrix = pd.DataFrame(data=table,columns = vec.get_feature_names_out())

    #print(memory_process_matrix.mean())
    memory_process_matrix = memory_process_matrix.add_prefix('mem_')
    for col in memory_process_matrix.columns:
        df[col] = memory_process_matrix[col]
    df.drop(['process_most_memory'],inplace=True,axis=1)
    
    return df

In [954]:
def ip_to_categorical(df):
    # IP Address - One-hot encoding
    data_ip = pd.get_dummies(df, columns=['ip_address'], 
                   drop_first=False,
                   dummy_na=False)
    df = data_ip
    return df

In [955]:
# Create categories for various throttled status
"""
Dropping due to time complexity (around 14s)

def get_throttled_status(binary):
    binary = str(binary)
    binary = binary.rjust(20)
    if binary[19]=='1':
        return 'under_voltage'
    elif binary[18]=='1':
        return 'throttled'
    elif binary[17]=='1':
        return 'arm_freq_capped'
    elif binary[16]=='1':
        return 'soft_temp_reached'
    else:
        return 'none'

df['throttled_status'] = df['throttled_status'].apply(lambda x: bin(int(x, 16))[2:])

throttled_categories = ['under_voltage','throttled','arm_freq_capped','soft_temp_reached']
throttled_columns = pd.DataFrame(0, index=np.arange(len(df)), columns=throttled_categories)
index=0
for status in df['throttled_status']:
    throttled_columns.iloc[index][get_throttled_status(status)]=1

"""

# Clean up GPIO pins
"""
Skipping due to over-complication 

df['gpio_pin']=df['gpio_pin'].apply(lambda x: list(x))

print(mlb.classes_)
mlb = MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(df['gpio_pin']),
                   columns=mlb.classes_,
                   index=df['gpio_pin'].index)
"""
print('Skip')

Skip


In [956]:
def string_to_categorical(df):
    # Convert or String datatypes in Categorical
    str_cols = [col for col in df.columns if df[col].dtype=='string']
    data_model = pd.get_dummies(df, columns=str_cols, 
                   drop_first=False,
                   dummy_na=False)
    df=data_model
    return df

In [957]:
def normal_validation_column(df):
    # Create column determining normal data (0) vs anomalous data (1)
    df = df.assign(anomalous=np.zeros((len(df)), dtype=int))
    return df

In [958]:
def get_features(df):
    return [f for f in list(df) if f not in ['anomalous']]

In [959]:
"""
Get Data
"""
df = clean_column_data_type(df)
# Drop datetime, boottime, uptime and throttled status (unsupported tensor format)
df = drop_columns(df,['time', 'boottime', 'uptime', 'throttled_status'])
df = vectorize_cpu_process(df)
df = vectorize_mem_process(df)
df = ip_to_categorical(df)
df = string_to_categorical(df)
df = normal_validation_column(df)

pd.set_option('display.max_columns', None)
print(len(df.columns))
print(df.head())

57
   voltage  ring_oscillation  cpu_temp  cpu%  cpu_count  system_calls  \
0    0.926             2.580    59.887  26.1          4             0   
1    0.926             2.580    61.348  34.5          4             0   
2    0.926             2.580    58.913   1.9          4             0   
3    0.926             2.580    60.861  11.7          4             0   
4    0.926             2.579    63.783  18.6          4             0   

   current_cpu_freq  min_cpu_freq  max_cpu_freq  system_load_avg_1min  \
0              1800           600          1800                 31.25   
1              1800           600          1800                 31.50   
2               600           600          1800                 24.50   
3              1800           600          1800                 18.00   
4              1800           600          1800                 26.00   

   system_load_avg_5min  system_load_avg_15min  physical_memory  ram_usage  \
0                 35.50                  

# Test/Train Data Split

In [849]:
# Random_state: you get same random split each time for reproducibility
# test_size: train/test split
# Both datasets are currently only made of normal data
X_train, X_test, Y_train, Y_test = train_test_split(df[get_features(df)],
                                                    df['anomalous'],
                                                    test_size=0.3, 
                                                    random_state=1)

print('Data split - Train:', len(X_train), 'Test:', len(X_test))
# check for nulls
X_train.isnull().values.sum()

Data split - Train: 4995 Test: 2142


0

# Standardize Data

In [903]:
# Feature Scaling of datasets
st_x= StandardScaler()  

X_train= st_x.fit_transform(X_train)  

X_test= st_x.transform(X_test) 

print(X_train.shape)
print(X_train[0])

(4995, 56)
[-0.39 -0.39 -0.36 -0.35  0.    0.   -0.39  0.    0.   -0.41 -0.44 -0.38
  0.    0.84  0.32 -0.31  0.64  0.    1.09  0.95  1.    0.95 -0.49 -0.5
 -0.01 -0.01 -0.06  0.   -0.11 -0.54 -0.05 -0.03 -0.09 -0.55 -0.01 -0.48
 -0.01  0.    0.    0.   -0.55 -0.14 -0.02 -0.97 -0.05  0.52  0.88 -0.16
 -0.27  0.32  0.    0.    0.    0.    0.    0.  ]


# Build and Train Model

In [771]:
# Build scaffolding
epochs = 500 # How many times to run model
batch_size= 100 # How many samples to feed each time
input_dim = df[get_features(df)].shape[1] # Input dimension
encoding_dim = 30 # Reduce input dimension
hidden_dim = int(encoding_dim/2) # Final reduction
learning_rate = 0.0001 # Too small may naver converge, too big it may converge too fast (.003)

# Build autoencoder
# Relu activation drops all negative numbers
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="relu", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(encoding_dim, activation="relu")(decoder)
decoder = Dense(input_dim, activation="linear")(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 56)]              0         
                                                                 
 dense_90 (Dense)            (None, 30)                1710      
                                                                 
 dense_91 (Dense)            (None, 15)                465       
                                                                 
 dense_92 (Dense)            (None, 15)                240       
                                                                 
 dense_93 (Dense)            (None, 30)                480       
                                                                 
 dense_94 (Dense)            (None, 56)                1736      
                                                                 
Total params: 4,631
Trainable params: 4,631
Non-trainable 

In [772]:
autoencoder.compile(metrics=['accuracy'], loss='mean_squared_error', optimizer='adam')
cp = ModelCheckpoint(filepath='autoencoder_classifier.h5', save_best_only=True, verbose=0)
tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

# No outcome variable so we pass same dataset twice
history = autoencoder.fit(X_train,
                          X_train,
                          epochs=epochs,
                          batch_size=batch_size,
                          shuffle=False, # Dont shuffle it is history dependent
                          validation_data=(X_test, X_test),
                          verbose=1,
                          callbacks=[cp, tb]).history

Epoch 1/500
50/50 [==============================] - 3s 24ms/step - loss: 0.7164 - accuracy: 0.0230 - val_loss: 0.6420 - val_accuracy: 0.0331
Epoch 2/500
50/50 [==============================] - 1s 14ms/step - loss: 0.6135 - accuracy: 0.0767 - val_loss: 0.5184 - val_accuracy: 0.0803
Epoch 3/500
50/50 [==============================] - 1s 15ms/step - loss: 0.4929 - accuracy: 0.1081 - val_loss: 0.4158 - val_accuracy: 0.2077
Epoch 4/500
50/50 [==============================] - 1s 16ms/step - loss: 0.4126 - accuracy: 0.3293 - val_loss: 0.3601 - val_accuracy: 0.3922
Epoch 5/500
50/50 [==============================] - 1s 15ms/step - loss: 0.3667 - accuracy: 0.4126 - val_loss: 0.3239 - val_accuracy: 0.4206
Epoch 6/500
50/50 [==============================] - 1s 18ms/step - loss: 0.3351 - accuracy: 0.4595 - val_loss: 0.2987 - val_accuracy: 0.5205
Epoch 7/500
50/50 [==============================] - 1s 21ms/step - loss: 0.3120 - accuracy: 0.5043 - val_loss: 0.2805 - val_accuracy: 0.5509
Epoch 

50/50 [==============================] - 1s 14ms/step - loss: 0.0580 - accuracy: 0.5712 - val_loss: 0.0615 - val_accuracy: 0.5714
Epoch 59/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0573 - accuracy: 0.5718 - val_loss: 0.0607 - val_accuracy: 0.5691
Epoch 60/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0562 - accuracy: 0.5654 - val_loss: 0.0602 - val_accuracy: 0.5733
Epoch 61/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0549 - accuracy: 0.5646 - val_loss: 0.0594 - val_accuracy: 0.5696
Epoch 62/500
50/50 [==============================] - 1s 15ms/step - loss: 0.0538 - accuracy: 0.5618 - val_loss: 0.0590 - val_accuracy: 0.5784
Epoch 63/500
50/50 [==============================] - 1s 14ms/step - loss: 0.0530 - accuracy: 0.5596 - val_loss: 0.0582 - val_accuracy: 0.5668
Epoch 64/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0521 - accuracy: 0.5578 - val_loss: 0.0576 - val_accuracy: 0.5714
Epoch 65/500

50/50 [==============================] - 1s 14ms/step - loss: 0.0400 - accuracy: 0.5986 - val_loss: 0.0449 - val_accuracy: 0.6074
Epoch 116/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0395 - accuracy: 0.5978 - val_loss: 0.0450 - val_accuracy: 0.6162
Epoch 117/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0396 - accuracy: 0.6000 - val_loss: 0.0444 - val_accuracy: 0.6148
Epoch 118/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0394 - accuracy: 0.5998 - val_loss: 0.0448 - val_accuracy: 0.6153
Epoch 119/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0398 - accuracy: 0.6032 - val_loss: 0.0445 - val_accuracy: 0.6134
Epoch 120/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0395 - accuracy: 0.6018 - val_loss: 0.0444 - val_accuracy: 0.6153
Epoch 121/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0397 - accuracy: 0.6058 - val_loss: 0.0446 - val_accuracy: 0.6246
Epoch 

Epoch 172/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0321 - accuracy: 0.6360 - val_loss: 0.0362 - val_accuracy: 0.6419
Epoch 173/500
50/50 [==============================] - 1s 14ms/step - loss: 0.0321 - accuracy: 0.6434 - val_loss: 0.0364 - val_accuracy: 0.6321
Epoch 174/500
50/50 [==============================] - 1s 22ms/step - loss: 0.0312 - accuracy: 0.6380 - val_loss: 0.0345 - val_accuracy: 0.6359
Epoch 175/500
50/50 [==============================] - 1s 17ms/step - loss: 0.0307 - accuracy: 0.6422 - val_loss: 0.0343 - val_accuracy: 0.6307
Epoch 176/500
50/50 [==============================] - 1s 23ms/step - loss: 0.0302 - accuracy: 0.6394 - val_loss: 0.0338 - val_accuracy: 0.6321
Epoch 177/500
50/50 [==============================] - 1s 16ms/step - loss: 0.0301 - accuracy: 0.6414 - val_loss: 0.0338 - val_accuracy: 0.6368
Epoch 178/500
50/50 [==============================] - 1s 17ms/step - loss: 0.0305 - accuracy: 0.6464 - val_loss: 0.0336 - val_accuracy:

Epoch 229/500
50/50 [==============================] - 1s 19ms/step - loss: 0.0268 - accuracy: 0.6593 - val_loss: 0.0281 - val_accuracy: 0.6522
Epoch 230/500
50/50 [==============================] - 1s 19ms/step - loss: 0.0260 - accuracy: 0.6721 - val_loss: 0.0276 - val_accuracy: 0.6419
Epoch 231/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0248 - accuracy: 0.6569 - val_loss: 0.0277 - val_accuracy: 0.6457
Epoch 232/500
50/50 [==============================] - 1s 17ms/step - loss: 0.0244 - accuracy: 0.6669 - val_loss: 0.0271 - val_accuracy: 0.6457
Epoch 233/500
50/50 [==============================] - 1s 15ms/step - loss: 0.0251 - accuracy: 0.6605 - val_loss: 0.0276 - val_accuracy: 0.6447
Epoch 234/500
50/50 [==============================] - 1s 14ms/step - loss: 0.0252 - accuracy: 0.6627 - val_loss: 0.0272 - val_accuracy: 0.6452
Epoch 235/500
50/50 [==============================] - 1s 15ms/step - loss: 0.0266 - accuracy: 0.6621 - val_loss: 0.0276 - val_accuracy:

Epoch 286/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0231 - accuracy: 0.6661 - val_loss: 0.0261 - val_accuracy: 0.6317
Epoch 287/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0247 - accuracy: 0.6717 - val_loss: 0.0263 - val_accuracy: 0.6447
Epoch 288/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0249 - accuracy: 0.6695 - val_loss: 0.0273 - val_accuracy: 0.6480
Epoch 289/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0258 - accuracy: 0.6739 - val_loss: 0.0268 - val_accuracy: 0.6415
Epoch 290/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0238 - accuracy: 0.6685 - val_loss: 0.0260 - val_accuracy: 0.6527
Epoch 291/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0240 - accuracy: 0.6691 - val_loss: 0.0260 - val_accuracy: 0.6461
Epoch 292/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0228 - accuracy: 0.6657 - val_loss: 0.0246 - val_accuracy:

Epoch 343/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0212 - accuracy: 0.6931 - val_loss: 0.0230 - val_accuracy: 0.6681
Epoch 344/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0208 - accuracy: 0.6951 - val_loss: 0.0226 - val_accuracy: 0.6816
Epoch 345/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0208 - accuracy: 0.6949 - val_loss: 0.0229 - val_accuracy: 0.6774
Epoch 346/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0203 - accuracy: 0.6931 - val_loss: 0.0225 - val_accuracy: 0.6835
Epoch 347/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0204 - accuracy: 0.6991 - val_loss: 0.0226 - val_accuracy: 0.6853
Epoch 348/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0202 - accuracy: 0.6977 - val_loss: 0.0224 - val_accuracy: 0.6769
Epoch 349/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0202 - accuracy: 0.6973 - val_loss: 0.0228 - val_accuracy:

Epoch 400/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0192 - accuracy: 0.7293 - val_loss: 0.0214 - val_accuracy: 0.6970
Epoch 401/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0193 - accuracy: 0.7283 - val_loss: 0.0212 - val_accuracy: 0.7021
Epoch 402/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0189 - accuracy: 0.7311 - val_loss: 0.0211 - val_accuracy: 0.6965
Epoch 403/500
50/50 [==============================] - 1s 14ms/step - loss: 0.0188 - accuracy: 0.7321 - val_loss: 0.0207 - val_accuracy: 0.7073
Epoch 404/500
50/50 [==============================] - 1s 11ms/step - loss: 0.0183 - accuracy: 0.7357 - val_loss: 0.0208 - val_accuracy: 0.7021
Epoch 405/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0187 - accuracy: 0.7317 - val_loss: 0.0211 - val_accuracy: 0.7068
Epoch 406/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0184 - accuracy: 0.7363 - val_loss: 0.0213 - val_accuracy:

Epoch 457/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0182 - accuracy: 0.7469 - val_loss: 0.0199 - val_accuracy: 0.7199
Epoch 458/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0178 - accuracy: 0.7467 - val_loss: 0.0196 - val_accuracy: 0.7138
Epoch 459/500
50/50 [==============================] - 1s 19ms/step - loss: 0.0179 - accuracy: 0.7475 - val_loss: 0.0196 - val_accuracy: 0.7269
Epoch 460/500
50/50 [==============================] - 1s 15ms/step - loss: 0.0177 - accuracy: 0.7463 - val_loss: 0.0197 - val_accuracy: 0.7152
Epoch 461/500
50/50 [==============================] - 1s 14ms/step - loss: 0.0175 - accuracy: 0.7457 - val_loss: 0.0194 - val_accuracy: 0.7218
Epoch 462/500
50/50 [==============================] - 1s 12ms/step - loss: 0.0173 - accuracy: 0.7461 - val_loss: 0.0195 - val_accuracy: 0.7166
Epoch 463/500
50/50 [==============================] - 1s 13ms/step - loss: 0.0173 - accuracy: 0.7497 - val_loss: 0.0193 - val_accuracy:

# Test Model

In [857]:
# Determine how well it is trained. Metric is mean squared error
test_x_predictions = autoencoder.predict(X_test) # Where live data in production is passed
print(test_x_predictions.shape)
mse = np.mean(np.power(X_test - test_x_predictions, 2), axis = 1) # If reconstruction error is zero it's norma
mse

(2142, 56)


array([0.02, 0.12, 0.01, ..., 0.  , 0.01, 0.  ])

In [898]:
# Random Testing Data
test_an = df[100:200]
test_an['cpu_temp']=test_anomaly['cpu_temp']
test_an.at[101,'cpu_temp'] = 100
print(test_an.loc[101]['cpu_temp'])
test_an = test_an[get_features(df)]
test_an= st_x.transform(test_an) 
test_an

100.0


/tmp/ipykernel_678327/854712637.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_an['cpu_temp']=test_anomaly['cpu_temp']


array([[-0.39, -0.4 ,  0.06, ...,  0.  ,  0.  ,  0.  ],
       [-0.39, -0.39,  7.34, ...,  0.  ,  0.  ,  0.  ],
       [-0.39, -0.39, -0.01, ...,  0.  ,  0.  ,  0.  ],
       ...,
       [-0.39, -0.39, -0.36, ...,  0.  ,  0.  ,  0.  ],
       [-0.39, -0.39, -0.36, ...,  0.  ,  0.  ,  0.  ],
       [-0.39, -0.4 , -0.36, ...,  0.  ,  0.  ,  0.  ]])

In [900]:
test_x1_predictions = autoencoder.predict(test_an) # Where live data in production is passed
print(test_x1_predictions.shape)
mse = np.mean(np.power(test_an - test_x1_predictions, 2), axis = 1) # If reconstruction error is zero it's norma
mse

(100, 56)


array([0.01, 0.75, 0.01, 0.01, 0.01, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.03, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.03, 0.01,
       0.01, 0.01, 0.01, 0.  , 0.01, 0.01, 0.  , 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.04, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01])